确保有od.csv(带站点清洗数据.py得到)

### 从od.csv中，按照工作日非工作日分别预测出0膨胀泊松分布

In [9]:
!pip install scipy
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 5.6 MB/s  0:00:02m0:00:0100:01


#### 预处理，生成0膨胀泊松分布的需求预测

In [6]:
import pandas as pd
import numpy as np
from scipy.stats import poisson

# ---- 读取 CSV ----
df = pd.read_csv("od.csv")

df['date'] = pd.to_datetime(df['date'])
df['start_station_id'] = df['start_station_id'].astype(str)
df['end_station_id'] = df['end_station_id'].astype(str)

# ---- 零膨胀泊松预测函数（保持你的原样） ----
def zip_predict(row):
    pi = row['zero_rate']         # 零膨胀率
    lambda_c = row['count_process_rate'] # 计数过程泊松率

    # 1. 结构性零
    if np.random.rand() < pi:
        return 0

    # 2. 泊松过程
    lam = max(lambda_c, 0)
    return np.random.poisson(lam)


# ====== 🚀 对 day_type=0 与 day_type=1 分别重复你的原流程 ======
for dt in sorted(df['day_type'].unique()):
    print(f"\n========== 开始处理 day_type = {dt} ==========")

    df_dt = df[df['day_type'] == dt].copy()

    # ---- 1. 统计天数 & 预处理 ----
    n_days = df_dt['date'].nunique()
    print("该 day_type 的历史天数 =", n_days)

    daily_trips = df_dt.groupby(
        ['date', 'hour', 'start_station_id', 'end_station_id']
    )['trips'].sum().reset_index()

    # ---- 2. 聚合统计 ----
    agg = daily_trips.groupby(['hour','start_station_id','end_station_id']).agg(
        total_trips=('trips','sum'),
        days_with_trips=('trips', lambda x: (x > 0).sum())
    ).reset_index()

    # 和你的代码完全一致
    agg['days_with_zero_trips'] = n_days - agg['days_with_trips']
    agg['zero_rate'] = agg['days_with_zero_trips'] / n_days      # π
    agg['overall_avg_trips'] = agg['total_trips'] / n_days       # λ

    # λ_c
    agg['count_process_rate'] = np.where(
        agg['zero_rate'] < 1.0,
        agg['overall_avg_trips'] / (1 - agg['zero_rate']),
        0.0
    )

    # ---- 3. ZIP 预测 ----
    agg['demand_avg'] = agg['overall_avg_trips']
    agg['demand_zip_poisson'] = agg.apply(zip_predict, axis=1)

    # ---- 4. ILP 需求整数（保持你的原版）----
    agg['demand_up'] = agg['overall_avg_trips'].apply(lambda x: int(x) if x.is_integer() else int(x) + 1)
    agg['demand_0.5'] = agg['overall_avg_trips'].round().astype(int)

    # ---- 5. 导出文件 ----
    out_name = f"od_hourly_zip_poisson_daytype_{dt}.csv"
    agg.to_csv(out_name, index=False)

    print(f"day_type={dt} 零膨胀泊松预测完成 → 已输出 {out_name}")
    print(agg[['hour', 'start_station_id', 'end_station_id', 'demand_avg', 'zero_rate', 'demand_zip_poisson']].head(10))

print("\n======= 全部 day_type ZIP 预测完成 =======")


========== 开始处理 day_type = 0 ==========
该 day_type 的历史天数 = 280
day_type=0 零膨胀泊松预测完成 → 已输出 od_hourly_zip_poisson_daytype_0.csv
   hour start_station_id end_station_id  demand_avg  zero_rate  \
0     0             1017           1024    0.003571   0.996429   
1     0             1017           1026    0.003571   0.996429   
2     0             1017           1039    0.003571   0.996429   
3     0             1017           1727    0.003571   0.996429   
4     0             1017           1814    0.003571   0.996429   
5     0             1017            189    0.003571   0.996429   
6     0             1017            253    0.003571   0.996429   
7     0             1017            265    0.003571   0.996429   
8     0             1017            290    0.003571   0.996429   
9     0             1019           1025    0.007143   0.992857   

   demand_zip_poisson  
0                   0  
1                   0  
2                   0  
3                   0  
4                   0  
5 

### 不分工作日和非工作日的（不必运行）

In [15]:
"""
最终版：用清洗好的 OD 平均需求运行多时段共享单车规划模型
依赖：pandas, numpy, pulp
"""

import pandas as pd
import numpy as np
import pulp
import math

# ================================
# 读取清洗后的 OD 数据
# ================================

#df = pd.read_csv("od_hourly_avg.csv")

#0膨胀poisson用下面这个
df = pd.read_csv("od_hourly_zip_poisson.csv")

# 确保 station 是字符串
df['start_station_id'] = df['start_station_id'].astype(str)
df['end_station_id']   = df['end_station_id'].astype(str)

# =================================
# 参数：成本（无需建站成本）
# =================================
r1 = 3     # 每辆车成本
r2 = 10    # 每个车位成本

# 选择需求字段： demand_up 或 demand_0.5
DEMAND_COL = "demand_zip_poisson"    # 或 "demand_up"

#DEMAND_COL = "demand_poisson"    

# ================================
# 整理 station index
# ================================
stations = sorted(pd.unique(pd.concat([df['start_station_id'], df['end_station_id']])))
N = len(stations)
station_to_idx = {z:i for i,z in enumerate(stations)}

df['i_idx'] = df['start_station_id'].map(station_to_idx)
df['j_idx'] = df['end_station_id'].map(station_to_idx)

# ================================
# 时段 = 自然小时（1..24）
# ================================
hours = sorted(df['hour'].unique())
K = max(hours)
print("使用自然小时 K =", K)

# ================================
# 构造 B[i,j,k]
# ================================
B = np.zeros((N, N, K+1), dtype=int)

for _, row in df.iterrows():
    i = row['i_idx']
    j = row['j_idx']
    k = int(row['hour'])
    demand = int(row[DEMAND_COL])
    if 1 <= k <= K:
        B[i,j,k] = demand

# ================================
# 构造旅行时间 t(i,j)
# 简化：1 小时内归还 → t = 1（或可设 >1）
# ================================
t = np.ones((N,N), dtype=int)

# ================================
# 计算 R[j,i,k] = B[j,i,k-t]
# ================================
R = np.zeros_like(B)

for i in range(N):
    for j in range(N):
        tij = t[i,j]
        for k in range(1, K+1):
            if k - tij > 0:
                R[i,j,k] = B[i,j,k-tij]

# ================================
# 构建 ILP 模型
# ================================
model = pulp.LpProblem("Bike_Planning", pulp.LpMinimize)

# x_i = initial bikes
# y_i = total parking capacity
xi = {i: pulp.LpVariable(f"x_{i}", lowBound=0, cat="Integer") for i in range(N)}
yi = {i: pulp.LpVariable(f"y_{i}", lowBound=0, cat="Integer") for i in range(N)}

# 初始最大停车位上界
MAX_PARK = 30    # 可改得更大

model += pulp.lpSum([r1 * xi[i] + r2 * yi[i] for i in range(N)])

# ================================
# 递推变量
# ================================
Vi = {}
Li = {}
A = {}
LA = {}

for i in range(N):
    for k in range(1, K+1):
        Vi[(i,k)] = pulp.LpVariable(f"V_{i}_{k}", lowBound=0, cat="Integer")
        Li[(i,k)] = pulp.LpVariable(f"L_{i}_{k}", lowBound=0, cat="Integer")
        A[(i,k)]  = pulp.LpVariable(f"A_{i}_{k}", lowBound=0, cat="Integer")
        LA[(i,k)] = pulp.LpVariable(f"LA_{i}_{k}", lowBound=0, cat="Integer")

# =============================================
# 约束（核心）
# =============================================

# 1) 初始 V_{i,1} = x_i
for i in range(N):
    model += Vi[(i,1)] == xi[i]

# 2) 递推 V
for i in range(N):
    for k in range(2, K+1):
        model += Vi[(i,k)] == \
            Vi[(i,k-1)] + \
            pulp.lpSum([R[j,i,k-1] for j in range(N)]) - \
            pulp.lpSum([B[i,j,k-1] for j in range(N)])

# 3) A = V + arrivals
for i in range(N):
    for k in range(1, K+1):
        model += A[(i,k)] == Vi[(i,k)] + pulp.lpSum([R[j,i,k] for j in range(N)])

# 4) L_i,k = y_i - V_i,k
for i in range(N):
    for k in range(1, K+1):
        model += Li[(i,k)] == yi[i] - Vi[(i,k)]

# 5) LA = free slots + departures（注意方向）
for i in range(N):
    for k in range(1, K+1):
        model += LA[(i,k)] == Li[(i,k)] + pulp.lpSum([B[i,j,k] for j in range(N)])

# 6) 可行性约束
#   (9) A_i,k >= 出发需求
for i in range(N):
    for k in range(1, K+1):
        model += A[(i,k)] >= pulp.lpSum([B[i,j,k] for j in range(N)])

#   (10) LA_i,k >= 到达需求
for i in range(N):
    for k in range(1, K+1):
        model += LA[(i,k)] >= pulp.lpSum([R[j,i,k] for j in range(N)])

# 7) x_i <= y_i
for i in range(N):
    model += xi[i] <= yi[i]

# 8) 容量上界
for i in range(N):
    model += yi[i] <= MAX_PARK

# ================================
# 求解
# ================================
solver = pulp.PULP_CBC_CMD(msg=False)
model.solve(solver)

# ----最优解结果 ----
total_x = sum(int(pulp.value(xi[i])) for i in range(N))
print("Total bikes (sum of x_i) =", total_x)
print("Status:", pulp.LpStatus[model.status])
print("Objective =", pulp.value(model.objective))

# 结果输出
res = pd.DataFrame({
    "station_id": stations,
    "x_i": [int(pulp.value(xi[i])) for i in range(N)],
    "y_i": [int(pulp.value(yi[i])) for i in range(N)]
})
res.to_csv("result_capacity.csv", index=False)
print(res.head())
print("已保存 result_capacity.csv")

使用自然小时 K = 23
Total bikes (sum of x_i) = 245
Status: Optimal
Objective = 5295.0
  station_id  x_i  y_i
0       1017    0    3
1       1018    0    0
2       1019    0    3
3       1024    2    6
4       1025    0    4
已保存 result_capacity.csv


### 区分了工作日与非工作日，分别生成了两份文档

#### （考虑了公平性问题，设置了容量下限）

In [19]:
"""
最终版：用清洗好的 OD 平均需求运行多时段共享单车规划模型
依赖：pandas, numpy, pulp
"""

import pandas as pd
import numpy as np
import pulp
import math

# ================================
# 封装成函数：给一个 OD 文件路径，输出一份 result_capacity 文件
# ================================
def run_model_for_file(input_csv, output_csv):
    print("\n=================================")
    print("运行模型：", input_csv)
    print("=================================")

    # ================================
    # 读取清洗后的 OD 数据
    # ================================
    df = pd.read_csv(input_csv)

    # 确保 station 是字符串
    df['start_station_id'] = df['start_station_id'].astype(str)
    df['end_station_id']   = df['end_station_id'].astype(str)

    # =================================
    # 参数：成本（无需建站成本）
    # =================================
    r1 = 3     # 每辆车成本
    r2 = 10    # 每个车位成本

    MAX_PARK = 30       #车站容量上限
    MIN_PARK = 3        #车站容量下限
    DEMAND_COL = "demand_zip_poisson"    # 使用 ZIP 预测结果字段

    # ================================
    # 整理 station index
    # ================================
    stations = sorted(pd.unique(pd.concat([df['start_station_id'], df['end_station_id']])))
    N = len(stations)
    station_to_idx = {z:i for i,z in enumerate(stations)}

    df['i_idx'] = df['start_station_id'].map(station_to_idx)
    df['j_idx'] = df['end_station_id'].map(station_to_idx)

    # ================================
    # 时段 = 自然小时（1..24）
    # ================================
    hours = sorted(df['hour'].unique())
    K = max(hours)
    print("使用自然小时 K =", K)
    print("需求总量=", df[DEMAND_COL].sum())
    # ================================
    # 构造 B[i,j,k]
    # ================================
    B = np.zeros((N, N, K+1), dtype=int)

    for _, row in df.iterrows():
        i = row['i_idx']
        j = row['j_idx']
        k = int(row['hour'])
        demand = int(row[DEMAND_COL])
        if 1 <= k <= K:
            B[i,j,k] = demand

    # ================================
    # 构造旅行时间 t(i,j)
    # ================================
    t = np.ones((N,N), dtype=int)

    # ================================
    # 计算 R[j,i,k] = B[j,i,k-t]
    # ================================
    R = np.zeros_like(B)

    for i in range(N):
        for j in range(N):
            tij = t[i,j]
            for k in range(1, K+1):
                if k - tij > 0:
                    R[i,j,k] = B[i,j,k-tij]

    # ================================
    # 构建 ILP 模型
    # ================================
    model = pulp.LpProblem("Bike_Planning", pulp.LpMinimize)

    # x_i = initial bikes
    # y_i = total parking capacity
    xi = {i: pulp.LpVariable(f"x_{i}", lowBound=0, cat="Integer") for i in range(N)}
    yi = {i: pulp.LpVariable(f"y_{i}", lowBound=0, cat="Integer") for i in range(N)}


    model += pulp.lpSum([r1 * xi[i] + r2 * yi[i] for i in range(N)])

    # ================================
    # 递推变量
    # ================================
    Vi = {}
    Li = {}
    A = {}
    LA = {}

    for i in range(N):
        for k in range(1, K+1):
            Vi[(i,k)] = pulp.LpVariable(f"V_{i}_{k}", lowBound=0, cat="Integer")
            Li[(i,k)] = pulp.LpVariable(f"L_{i}_{k}", lowBound=0, cat="Integer")
            A[(i,k)]  = pulp.LpVariable(f"A_{i}_{k}", lowBound=0, cat="Integer")
            LA[(i,k)] = pulp.LpVariable(f"LA_{i}_{k}", lowBound=0, cat="Integer")

    # =============================================
    # 约束（核心）
    # =============================================

    # 1) 初始 V_{i,1} = x_i
    for i in range(N):
        model += Vi[(i,1)] == xi[i]

    # 2) 递推 V
    for i in range(N):
        for k in range(2, K+1):
            model += Vi[(i,k)] == \
                Vi[(i,k-1)] + \
                pulp.lpSum([R[j,i,k-1] for j in range(N)]) - \
                pulp.lpSum([B[i,j,k-1] for j in range(N)])

    # 3) A = V + arrivals
    for i in range(N):
        for k in range(1, K+1):
            model += A[(i,k)] == Vi[(i,k)] + pulp.lpSum([R[j,i,k] for j in range(N)])

    # 4) L_i,k = y_i - V_i,k
    for i in range(N):
        for k in range(1, K+1):
            model += Li[(i,k)] == yi[i] - Vi[(i,k)]

    # 5) LA = free slots + departures
    for i in range(N):
        for k in range(1, K+1):
            model += LA[(i,k)] == Li[(i,k)] + pulp.lpSum([B[i,j,k] for j in range(N)])

    # 6) 可行性约束
    for i in range(N):
        for k in range(1, K+1):
            model += A[(i,k)]  >= pulp.lpSum([B[i,j,k] for j in range(N)])
            model += LA[(i,k)] >= pulp.lpSum([R[j,i,k] for j in range(N)])

    # 7) x_i <= y_i
    for i in range(N):
        model += xi[i] <= yi[i]

    # 8) 容量上下界
    for i in range(N):
        model += yi[i] <= MAX_PARK
        model += yi[i] >= MIN_PARK
    # ================================
    # 求解
    # ================================
    solver = pulp.PULP_CBC_CMD(msg=False)
    model.solve(solver)

    print("Total bikes (sum of x_i) =", total_x)
    print("Status:", pulp.LpStatus[model.status])
    print("Objective =", pulp.value(model.objective))

    # ================================
    # 导出结果
    # ================================
    res = pd.DataFrame({
        "station_id": stations,
        "x_i": [int(pulp.value(xi[i])) for i in range(N)],
        "y_i": [int(pulp.value(yi[i])) for i in range(N)]
    })
    res.to_csv(output_csv, index=False)
    print(res.head())
    print("已保存", output_csv)


# =====================================================
# 对 daytype 0 和 1 两类文件分别运行
# =====================================================
run_model_for_file(
    "od_hourly_zip_poisson_daytype_0.csv",
    "Minlimit_capacity_daytype_0.csv"
)

run_model_for_file(
    "od_hourly_zip_poisson_daytype_1.csv",
    "Minlimit_capacity_daytype_1.csv"
)


运行模型： od_hourly_zip_poisson_daytype_0.csv
使用自然小时 K = 23
需求总量= 379
Total bikes (sum of x_i) = 245
Status: Optimal
Objective = 8302.0
  station_id  x_i  y_i
0       1017    2    3
1       1018    0    3
2       1019    4    4
3       1024    0    6
4       1025    5    5
已保存 Minlimit_capacity_daytype_0.csv

运行模型： od_hourly_zip_poisson_daytype_1.csv
使用自然小时 K = 23
需求总量= 355
Total bikes (sum of x_i) = 245
Status: Optimal
Objective = 7895.0
  station_id  x_i  y_i
0       1017    1    3
1       1018    0    3
2       1019    6    7
3       1024    4    6
4       1025    9    9
已保存 Minlimit_capacity_daytype_1.csv


#### 不考虑容量下限（忽略公平性，对站点进行优化）

In [20]:
"""
最终版：用清洗好的 OD 平均需求运行多时段共享单车规划模型
依赖：pandas, numpy, pulp
"""

import pandas as pd
import numpy as np
import pulp
import math

# ================================
# 封装成函数：给一个 OD 文件路径，输出一份 result_capacity 文件
# ================================
def run_model_for_file(input_csv, output_csv):
    print("\n=================================")
    print("运行模型：", input_csv)
    print("=================================")

    # ================================
    # 读取清洗后的 OD 数据
    # ================================
    df = pd.read_csv(input_csv)

    # 确保 station 是字符串
    df['start_station_id'] = df['start_station_id'].astype(str)
    df['end_station_id']   = df['end_station_id'].astype(str)

    # =================================
    # 参数：成本（无需建站成本）
    # =================================
    r1 = 3     # 每辆车成本
    r2 = 10    # 每个车位成本

    MAX_PARK = 30       #车站容量上限
    DEMAND_COL = "demand_zip_poisson"    # 使用 ZIP 预测结果字段

    # ================================
    # 整理 station index
    # ================================
    stations = sorted(pd.unique(pd.concat([df['start_station_id'], df['end_station_id']])))
    N = len(stations)
    station_to_idx = {z:i for i,z in enumerate(stations)}

    df['i_idx'] = df['start_station_id'].map(station_to_idx)
    df['j_idx'] = df['end_station_id'].map(station_to_idx)

    # ================================
    # 时段 = 自然小时（1..24）
    # ================================
    hours = sorted(df['hour'].unique())
    K = max(hours)
    print("使用自然小时 K =", K)
    print("需求总量=", df[DEMAND_COL].sum())
    # ================================
    # 构造 B[i,j,k]
    # ================================
    B = np.zeros((N, N, K+1), dtype=int)

    for _, row in df.iterrows():
        i = row['i_idx']
        j = row['j_idx']
        k = int(row['hour'])
        demand = int(row[DEMAND_COL])
        if 1 <= k <= K:
            B[i,j,k] = demand

    # ================================
    # 构造旅行时间 t(i,j)
    # ================================
    t = np.ones((N,N), dtype=int)

    # ================================
    # 计算 R[j,i,k] = B[j,i,k-t]
    # ================================
    R = np.zeros_like(B)

    for i in range(N):
        for j in range(N):
            tij = t[i,j]
            for k in range(1, K+1):
                if k - tij > 0:
                    R[i,j,k] = B[i,j,k-tij]

    # ================================
    # 构建 ILP 模型
    # ================================
    model = pulp.LpProblem("Bike_Planning", pulp.LpMinimize)

    # x_i = initial bikes
    # y_i = total parking capacity
    xi = {i: pulp.LpVariable(f"x_{i}", lowBound=0, cat="Integer") for i in range(N)}
    yi = {i: pulp.LpVariable(f"y_{i}", lowBound=0, cat="Integer") for i in range(N)}


    model += pulp.lpSum([r1 * xi[i] + r2 * yi[i] for i in range(N)])

    # ================================
    # 递推变量
    # ================================
    Vi = {}
    Li = {}
    A = {}
    LA = {}

    for i in range(N):
        for k in range(1, K+1):
            Vi[(i,k)] = pulp.LpVariable(f"V_{i}_{k}", lowBound=0, cat="Integer")
            Li[(i,k)] = pulp.LpVariable(f"L_{i}_{k}", lowBound=0, cat="Integer")
            A[(i,k)]  = pulp.LpVariable(f"A_{i}_{k}", lowBound=0, cat="Integer")
            LA[(i,k)] = pulp.LpVariable(f"LA_{i}_{k}", lowBound=0, cat="Integer")

    # =============================================
    # 约束（核心）
    # =============================================

    # 1) 初始 V_{i,1} = x_i
    for i in range(N):
        model += Vi[(i,1)] == xi[i]

    # 2) 递推 V
    for i in range(N):
        for k in range(2, K+1):
            model += Vi[(i,k)] == \
                Vi[(i,k-1)] + \
                pulp.lpSum([R[j,i,k-1] for j in range(N)]) - \
                pulp.lpSum([B[i,j,k-1] for j in range(N)])

    # 3) A = V + arrivals
    for i in range(N):
        for k in range(1, K+1):
            model += A[(i,k)] == Vi[(i,k)] + pulp.lpSum([R[j,i,k] for j in range(N)])

    # 4) L_i,k = y_i - V_i,k
    for i in range(N):
        for k in range(1, K+1):
            model += Li[(i,k)] == yi[i] - Vi[(i,k)]

    # 5) LA = free slots + departures
    for i in range(N):
        for k in range(1, K+1):
            model += LA[(i,k)] == Li[(i,k)] + pulp.lpSum([B[i,j,k] for j in range(N)])

    # 6) 可行性约束
    for i in range(N):
        for k in range(1, K+1):
            model += A[(i,k)]  >= pulp.lpSum([B[i,j,k] for j in range(N)])
            model += LA[(i,k)] >= pulp.lpSum([R[j,i,k] for j in range(N)])

    # 7) x_i <= y_i
    for i in range(N):
        model += xi[i] <= yi[i]

    # 8) 容量上下界
    for i in range(N):
        model += yi[i] <= MAX_PARK
    # ================================
    # 求解
    # ================================
    solver = pulp.PULP_CBC_CMD(msg=False)
    model.solve(solver)

    print("Total bikes (sum of x_i) =", total_x)
    print("Status:", pulp.LpStatus[model.status])
    print("Objective =", pulp.value(model.objective))

    # ================================
    # 导出结果
    # ================================
    res = pd.DataFrame({
        "station_id": stations,
        "x_i": [int(pulp.value(xi[i])) for i in range(N)],
        "y_i": [int(pulp.value(yi[i])) for i in range(N)]
    })
    res.to_csv(output_csv, index=False)
    print(res.head())
    print("已保存", output_csv)


# =====================================================
# 对 daytype 0 和 1 两类文件分别运行
# =====================================================
run_model_for_file(
    "od_hourly_zip_poisson_daytype_0.csv",
    "Nomin_capacity_daytype_0.csv"
)

run_model_for_file(
    "od_hourly_zip_poisson_daytype_1.csv",
    "Nomin_capacity_daytype_1.csv"
)


运行模型： od_hourly_zip_poisson_daytype_0.csv
使用自然小时 K = 23
需求总量= 379
Total bikes (sum of x_i) = 245
Status: Optimal
Objective = 5432.0
  station_id  x_i  y_i
0       1017    2    3
1       1018    0    0
2       1019    4    4
3       1024    0    6
4       1025    5    5
已保存 Nomin_capacity_daytype_0.csv

运行模型： od_hourly_zip_poisson_daytype_1.csv
使用自然小时 K = 23
需求总量= 355
Total bikes (sum of x_i) = 245
Status: Optimal
Objective = 4925.0
  station_id  x_i  y_i
0       1017    1    2
1       1018    0    0
2       1019    6    7
3       1024    4    6
4       1025    9    9
已保存 Nomin_capacity_daytype_1.csv


## 2025.11.17更新： 以下内容还未调试好

### 对工作日和非工作日的站点容量，取最大值作为站点容量，计算此时的成本

#### 对有容量下限的

In [23]:
import pandas as pd

# 读取两个文件
df0 = pd.read_csv("Minlimit_capacity_daytype_0.csv")
df1 = pd.read_csv("Minlimit_capacity_daytype_1.csv")

# 合并（不改变两个原 DataFrame）
dfm = df0.merge(df1, on="station_id", suffixes=("_0", "_1"))

# 求 y_i 最大值
dfm["y_i_max"] = dfm[["y_i_0", "y_i_1"]].max(axis=1)

# ==========================
# 生成 daytype_0 的输出
# ==========================
df0_out = df0.copy()
df0_out = df0_out.merge(dfm[["station_id", "y_i_max"]], on="station_id")
df0_out = df0_out.drop(columns=["y_i"])
df0_out = df0_out.rename(columns={"y_i_max": "y_i"})

df0_out.to_csv("result_capacity_daytype_0_with_yimax.csv", index=False)
print("已生成 result_capacity_daytype_0_with_yimax.csv")

# ==========================
# 生成 daytype_1 的输出
# ==========================
df1_out = df1.copy()
df1_out = df1_out.merge(dfm[["station_id", "y_i_max"]], on="station_id")
df1_out = df1_out.drop(columns=["y_i"])
df1_out = df1_out.rename(columns={"y_i_max": "y_i"})

df1_out.to_csv("result_capacity_daytype_1_with_yimax.csv", index=False)
print("已生成 result_capacity_daytype_1_with_yimax.csv")

# ==========================
# 统计 x_i 和 y_i 的数量
# ==========================

# daytype 0
x0_sum = df0_out["x_i"].sum()
y0_sum = df0_out["y_i"].sum()

print(f"Daytype 0：总 x_i = {x0_sum}, 总 y_i = {y0_sum}")

# daytype 1
x1_sum = df1_out["x_i"].sum()
y1_sum = df1_out["y_i"].sum()

print(f"Daytype 1：总 x_i = {x1_sum}, 总 y_i = {y1_sum}")

已生成 result_capacity_daytype_0_with_yimax.csv
已生成 result_capacity_daytype_1_with_yimax.csv
Daytype 0：总 x_i = 254, 总 y_i = 824
Daytype 1：总 x_i = 225, 总 y_i = 824


In [ ]:
# ==========================
# 统计 x_i 和 y_i 的数量
# ==========================

# daytype 0
x0_sum = df0_out["x_i"].sum()
y0_sum = df0_out["y_i"].sum()

print(f"Daytype 0：总 x_i = {x0_sum}, 总 y_i = {y0_sum}")

# daytype 1
x1_sum = df1_out["x_i"].sum()
y1_sum = df1_out["y_i"].sum()

print(f"Daytype 1：总 x_i = {x1_sum}, 总 y_i = {y1_sum}")